In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error

In [40]:
d1=pd.read_csv("../DataFrom2011.csv", parse_dates=True)
d1['Date']=pd.to_datetime(d1['Date'])
d1.info()
d1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13240 entries, 0 to 13239
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   StationID  13240 non-null  int64         
 1   Date       13240 non-null  datetime64[ns]
 2   Tmax       13240 non-null  float64       
 3   Tmin       13240 non-null  float64       
 4   Tdew       13240 non-null  float64       
 5   RHav       13240 non-null  float64       
 6   Tav        13240 non-null  float64       
 7   Rs         13240 non-null  float64       
 8   ws         13240 non-null  float64       
 9   PM         13240 non-null  float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 1.0 MB


,StationID,Date,Tmax,Tmin,Tdew,RHav,Tav,Rs,ws,PM
0,250,2011-01-01,15.62,6.97,12.283542,74.5385,11.295,10.664181,0.523110,1.750236
1,260,2011-01-01,15.34,10.19,13.218042,73.7415,12.765,10.513628,1.229216,1.984740
2,440,2011-01-01,18.53,12.49,16.412042,79.4950,15.510,9.999696,1.795443,2.100595
3,450,2011-01-01,17.76,10.48,14.908790,76.5100,14.120,11.450830,1.397206,2.189690
4,250,2011-01-02,18.64,11.67,16.324250,80.2500,15.155,6.566224,1.225372,1.614004
...,...,...,...,...,...,...,...,...,...,...
13235,260,2020-12-30,17.57,7.81,13.685833,78.7150,12.690,9.839592,1.012131,1.836750
13236,450,2020-12-30,19.32,14.05,16.840080,78.0075,16.685,8.546898,1.778988,2.060486
13237,250,2020-12-31,20.10,16.72,18.507083,79.1300,18.410,7.740450,1.487246,1.932808
13238,260,2020-12-31,19.67,13.68,18.006250,81.8450,16.675,7.599645,1.473050,1.788100


In [41]:
p = d1.drop(columns=['Date'],axis=1)
p = p.drop(columns=['StationID'],axis=1)
scaler = preprocessing.MinMaxScaler()
names = p.columns
d = scaler.fit_transform(p)
scaled_d1 = pd.DataFrame(d, columns=names)
scaled_d1.head()

,Tmax,Tmin,Tdew,RHav,Tav,Rs,ws,PM
0,0.367468,0.378016,0.414139,0.471390,0.370632,0.347008,0.236408,0.260056
1,0.356440,0.485925,0.452886,0.452151,0.426771,0.342109,0.555517,0.311915
2,0.482080,0.563003,0.585317,0.591033,0.531602,0.325386,0.811410,0.337536
3,0.451753,0.495643,0.522988,0.518979,0.478518,0.372605,0.631436,0.357238
4,0.486412,0.535523,0.581677,0.609257,0.518045,0.213662,0.553780,0.229929


In [42]:
d1 = pd.concat([d1['Date'], scaled_d1], axis=1) 
d1

,Date,Tmax,Tmin,Tdew,RHav,Tav,Rs,ws,PM
0,2011-01-01,0.367468,0.378016,0.414139,0.471390,0.370632,0.347008,0.236408,0.260056
1,2011-01-01,0.356440,0.485925,0.452886,0.452151,0.426771,0.342109,0.555517,0.311915
2,2011-01-01,0.482080,0.563003,0.585317,0.591033,0.531602,0.325386,0.811410,0.337536
3,2011-01-01,0.451753,0.495643,0.522988,0.518979,0.478518,0.372605,0.631436,0.357238
4,2011-01-02,0.486412,0.535523,0.581677,0.609257,0.518045,0.213662,0.553780,0.229929
...,...,...,...,...,...,...,...,...,...
13235,2020-12-30,0.444269,0.406166,0.472281,0.572204,0.423907,0.320176,0.457410,0.279188
13236,2020-12-30,0.513194,0.615282,0.603064,0.555126,0.576475,0.278113,0.803974,0.328666
13237,2020-12-31,0.543915,0.704759,0.672182,0.582222,0.642352,0.251871,0.672128,0.300431
13238,2020-12-31,0.526979,0.602882,0.651417,0.647758,0.576093,0.247289,0.665712,0.268429


In [43]:
d1 = d1.set_index(['Date'])
d1

,Tmax,Tmin,Tdew,RHav,Tav,Rs,ws,PM
Date,,,,,,,,
2011-01-01,0.367468,0.378016,0.414139,0.471390,0.370632,0.347008,0.236408,0.260056
2011-01-01,0.356440,0.485925,0.452886,0.452151,0.426771,0.342109,0.555517,0.311915
2011-01-01,0.482080,0.563003,0.585317,0.591033,0.531602,0.325386,0.811410,0.337536
2011-01-01,0.451753,0.495643,0.522988,0.518979,0.478518,0.372605,0.631436,0.357238
2011-01-02,0.486412,0.535523,0.581677,0.609257,0.518045,0.213662,0.553780,0.229929
...,...,...,...,...,...,...,...,...
2020-12-30,0.444269,0.406166,0.472281,0.572204,0.423907,0.320176,0.457410,0.279188
2020-12-30,0.513194,0.615282,0.603064,0.555126,0.576475,0.278113,0.803974,0.328666
2020-12-31,0.543915,0.704759,0.672182,0.582222,0.642352,0.251871,0.672128,0.300431


In [44]:
predictors=d1.drop(columns=['PM','Tmin','Tdew','ws'])
predictors

,Tmax,RHav,Tav,Rs
Date,,,,
2011-01-01,0.367468,0.471390,0.370632,0.347008
2011-01-01,0.356440,0.452151,0.426771,0.342109
2011-01-01,0.482080,0.591033,0.531602,0.325386
2011-01-01,0.451753,0.518979,0.478518,0.372605
2011-01-02,0.486412,0.609257,0.518045,0.213662
...,...,...,...,...
2020-12-30,0.444269,0.572204,0.423907,0.320176
2020-12-30,0.513194,0.555126,0.576475,0.278113
2020-12-31,0.543915,0.582222,0.642352,0.251871


In [45]:
target=d1['PM']
target

Date
2011-01-01    0.260056
2011-01-01    0.311915
2011-01-01    0.337536
2011-01-01    0.357238
2011-01-02    0.229929
                ...   
2020-12-30    0.279188
2020-12-30    0.328666
2020-12-31    0.300431
2020-12-31    0.268429
2020-12-31    0.463428
Name: PM, Length: 13240, dtype: float64

In [46]:
predictors=np.array(predictors)
target=np.array(target)
nobs = 2635
train_features, test_features = predictors[0:-nobs], predictors[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]

# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 4)
(10605,)
(2635, 4)
(2635,)


In [48]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15,18,21],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[13:58:07] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 1000, 'regressor__max_depth': 15, 'regressor__learning_rate': 0.05}


In [49]:
best_pipe=random_cv.best_estimator_

In [50]:
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))

MAE=  0.018923703408331213
MAPE=  0.04740696512975932
MSE=  0.0006438678229327135
R^2 train: 0.9810587269808738


In [22]:
ypred1

array([0.36494836, 0.33724764, 0.35989356, ..., 0.26429367, 0.25100222,
       0.42657813], dtype=float32)

In [23]:
test_labels

array([0.32676723, 0.32187252, 0.36438484, ..., 0.3004307 , 0.26842943,
       0.46342814])

In [51]:
import joblib
joblib.dump(best_pipe, ".XGB1.joblib")

['.XGB1.joblib']

In [ ]:
#########################################################################################################

In [ ]:
############################ model 2 Tmax, RHav, Tav ###############################################

In [52]:
predictors2=d1.drop(columns=['PM','Tdew','ws','Tmin','Rs'])
predictors2

,Tmax,RHav,Tav
Date,,,
2011-01-01,0.367468,0.471390,0.370632
2011-01-01,0.356440,0.452151,0.426771
2011-01-01,0.482080,0.591033,0.531602
2011-01-01,0.451753,0.518979,0.478518
2011-01-02,0.486412,0.609257,0.518045
...,...,...,...
2020-12-30,0.444269,0.572204,0.423907
2020-12-30,0.513194,0.555126,0.576475
2020-12-31,0.543915,0.582222,0.642352


In [53]:
predictors2=np.array(predictors2)
nobs = 2635
train_features, test_features = predictors2[0:-nobs], predictors2[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 3)
(10605,)
(2635, 3)
(2635,)


In [55]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000,7000,8000],
                    'regressor__max_depth':[3,6,9,12,15,18,21],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[14:00:25] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 4000, 'regressor__max_depth': 9, 'regressor__learning_rate': 0.1}


In [56]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB2.joblib")

MAE=  0.07281827746938105
MAPE=  0.20879724694466517
MSE=  0.00911404448626849
R^2 train: 0.7318834723925723


['.XGB2.joblib']

In [ ]:
#########################################################################################################

In [ ]:
####################### model 3 Tmax, RHav, Rs ###############################################

In [57]:
predictors3=d1.drop(columns=['PM','Tdew','ws','Tmin','Tav'])
predictors3

,Tmax,RHav,Rs
Date,,,
2011-01-01,0.367468,0.471390,0.347008
2011-01-01,0.356440,0.452151,0.342109
2011-01-01,0.482080,0.591033,0.325386
2011-01-01,0.451753,0.518979,0.372605
2011-01-02,0.486412,0.609257,0.213662
...,...,...,...
2020-12-30,0.444269,0.572204,0.320176
2020-12-30,0.513194,0.555126,0.278113
2020-12-31,0.543915,0.582222,0.251871


In [31]:
predictors3=np.array(predictors3)
nobs = 2635
train_features, test_features = predictors3[0:-nobs], predictors3[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 3)
(10605,)
(2635, 3)
(2635,)


In [32]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[12:01:22] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 6000, 'regressor__max_depth': 12, 'regressor__learning_rate': 0.05}


In [58]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB3.joblib")

MAE=  0.07281827746938105
MAPE=  0.20879724694466517
MSE=  0.00911404448626849
R^2 train: 0.7318834723925723


['.XGB3.joblib']

In [ ]:
#############################################################################################################

In [ ]:
############################# model 4 Tmax, Tav, Rs #############################

In [61]:
predictors4=d1.drop(columns=['PM','Tdew','ws','Tmin','RHav'])
predictors4

,Tmax,Tav,Rs
Date,,,
2011-01-01,0.367468,0.370632,0.347008
2011-01-01,0.356440,0.426771,0.342109
2011-01-01,0.482080,0.531602,0.325386
2011-01-01,0.451753,0.478518,0.372605
2011-01-02,0.486412,0.518045,0.213662
...,...,...,...
2020-12-30,0.444269,0.423907,0.320176
2020-12-30,0.513194,0.576475,0.278113
2020-12-31,0.543915,0.642352,0.251871


In [62]:
predictors4=np.array(predictors4)
nobs = 2635
train_features, test_features = predictors4[0:-nobs], predictors4[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 3)
(10605,)
(2635, 3)
(2635,)


In [63]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[14:08:21] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 500, 'regressor__max_depth': 6, 'regressor__learning_rate': 0.03}


In [64]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB4.joblib")

MAE=  0.0208698307226225
MAPE=  0.05560542969014669
MSE=  0.0007627435266154
R^2 train: 0.9775616471787825


['.XGB4.joblib']

In [ ]:
######################################################################################################

In [ ]:
########## model 5 Tmax, RHav ###################################################################

In [67]:
predictors5=d1.drop(columns=['PM','Tdew','ws','Tmin','Rs','Tav'])
predictors5

,Tmax,RHav
Date,,
2011-01-01,0.367468,0.471390
2011-01-01,0.356440,0.452151
2011-01-01,0.482080,0.591033
2011-01-01,0.451753,0.518979
2011-01-02,0.486412,0.609257
...,...,...
2020-12-30,0.444269,0.572204
2020-12-30,0.513194,0.555126
2020-12-31,0.543915,0.582222


In [68]:
predictors5=np.array(predictors5)
nobs = 2635
train_features, test_features = predictors5[0:-nobs], predictors5[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 


(10605, 2)
(10605,)
(2635, 2)
(2635,)


In [69]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[14:12:14] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 5000, 'regressor__max_depth': 6, 'regressor__learning_rate': 0.05}


In [70]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB5.joblib")

MAE=  0.07609526830689588
MAPE=  0.2110751770916706
MSE=  0.009879032552804924
R^2 train: 0.709379089802616


['.XGB5.joblib']

In [ ]:
#############################################################################################################

In [ ]:
######## model 6 Tmax, Tav ############################################

In [71]:
predictors6=d1.drop(columns=['PM','Tdew','ws','Tmin','Rs','RHav'])
predictors6

,Tmax,Tav
Date,,
2011-01-01,0.367468,0.370632
2011-01-01,0.356440,0.426771
2011-01-01,0.482080,0.531602
2011-01-01,0.451753,0.478518
2011-01-02,0.486412,0.518045
...,...,...
2020-12-30,0.444269,0.423907
2020-12-30,0.513194,0.576475
2020-12-31,0.543915,0.642352


In [72]:
predictors6=np.array(predictors6)
nobs = 2635
train_features, test_features = predictors6[0:-nobs], predictors6[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 2)
(10605,)
(2635, 2)
(2635,)


In [73]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)


[14:21:20] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 1000, 'regressor__max_depth': 15, 'regressor__learning_rate': 0.2}


In [74]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB6.joblib")

MAE=  0.13459756707172377
MAPE=  0.448543839721739
MSE=  0.02907525441780006
R^2 train: 0.14466554716206337


['.XGB6.joblib']

In [ ]:
#######################################################################################################

In [ ]:
########## model 7 Tmax, Rs ############################

In [75]:
predictors7=d1.drop(columns=['PM','Tdew','ws','Tmin','RHav','Tav'])
predictors7

,Tmax,Rs
Date,,
2011-01-01,0.367468,0.347008
2011-01-01,0.356440,0.342109
2011-01-01,0.482080,0.325386
2011-01-01,0.451753,0.372605
2011-01-02,0.486412,0.213662
...,...,...
2020-12-30,0.444269,0.320176
2020-12-30,0.513194,0.278113
2020-12-31,0.543915,0.251871


In [76]:
predictors7=np.array(predictors7)
nobs = 2635
train_features, test_features = predictors7[0:-nobs], predictors7[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape)

(10605, 2)
(10605,)
(2635, 2)
(2635,)


In [77]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[14:36:18] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 5000, 'regressor__max_depth': 9, 'regressor__learning_rate': 0.2}


In [78]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGB7.joblib")

MAE=  0.021957608075925155
MAPE=  0.056338061342943994
MSE=  0.0008301271559330995
R^2 train: 0.9755793587735122


['.XGB7.joblib']

In [ ]:
#####################################################################################################

In [ ]:
######### model ws on Tmax, RHav, Rs 

In [79]:
predictorsf=d1.drop(columns=['PM','Tdew','Tav','Tmin'])
predictorsf

,Tmax,RHav,Rs,ws
Date,,,,
2011-01-01,0.367468,0.471390,0.347008,0.236408
2011-01-01,0.356440,0.452151,0.342109,0.555517
2011-01-01,0.482080,0.591033,0.325386,0.811410
2011-01-01,0.451753,0.518979,0.372605,0.631436
2011-01-02,0.486412,0.609257,0.213662,0.553780
...,...,...,...,...
2020-12-30,0.444269,0.572204,0.320176,0.457410
2020-12-30,0.513194,0.555126,0.278113,0.803974
2020-12-31,0.543915,0.582222,0.251871,0.672128


In [80]:
predictorsf=np.array(predictorsf)
nobs = 2635
train_features, test_features = predictorsf[0:-nobs], predictorsf[-nobs:]
train_labels, test_labels = target[0:-nobs], target[-nobs:]
# Check size
print(train_features.shape)  
print(train_labels.shape)  
print(test_features.shape)  
print(test_labels.shape) 

(10605, 4)
(10605,)
(2635, 4)
(2635,)


In [86]:
hyperparameter_grid={'regressor__n_estimators':[100,500,1000,2000,3000,4000,5000,6000,7000],
                    'regressor__max_depth':[3,6,9,12,15],
                    'regressor__learning_rate':[0.01,0.03,0.05,0.1,0.2,0.5]}

random_cv=RandomizedSearchCV(XGBRegressor(), param_distributions=hyperparameter_grid,cv=3,n_iter=15,
                           scoring='neg_root_mean_squared_error',n_jobs=-1)
random_cv.fit(train_features,train_labels)
print(random_cv.best_params_)

[14:51:06] WARNING: ..\src\learner.cc:576: 
Parameters: { "regressor__learning_rate", "regressor__max_depth", "regressor__n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


{'regressor__n_estimators': 6000, 'regressor__max_depth': 12, 'regressor__learning_rate': 0.01}


In [87]:
best_pipe=random_cv.best_estimator_
ypred1=best_pipe.predict(test_features)
print("MAE= ", mean_absolute_error(test_labels, ypred1))
print("MAPE= ",mean_absolute_percentage_error(test_labels, ypred1))
print("MSE= ",mean_squared_error(test_labels, ypred1))
print('R^2 train:', r2_score(test_labels, ypred1))
joblib.dump(best_pipe, ".XGBf.joblib")

MAE=  0.010560737177836904
MAPE=  0.02803534985857692
MSE=  0.00019453716142476553
R^2 train: 0.9942771150294035


['.XGBf.joblib']